In [1]:
from typing import Any, Dict, Iterator, List, Optional

from langchain_core.callbacks import (
    CallbackManagerForLLMRun,
)
from langchain_core.language_models import BaseChatModel
from langchain_core.messages import (
    AIMessage,
    AIMessageChunk,
    BaseMessage,
    HumanMessage,
    
)
from langchain_core.messages.ai import UsageMetadata
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult
from pydantic import Field

## CustomLLM

In [ ]:
from langchain_openai import ChatOpenAI

# 한글 테스트 이거슨 malgun gothic

In [30]:
from typing import Callable, Literal, Sequence, Union
from langchain_core.tools import BaseTool
from langchain_core.runnables import Runnable
from langchain_core.language_models import LanguageModelInput
from langchain_core.utils.function_calling import (
    convert_to_openai_function,
    convert_to_openai_tool,
)


class ChatParrotLink(BaseChatModel):
    
    model_name: str = Field(alias="model")
    """The name of the model"""
    parrot_buffer_length: int
    """The number of characters from the last message of the prompt to be echoed."""
    temperature: Optional[float] = None
    max_tokens: Optional[int] = None
    timeout: Optional[int] = None
    stop: Optional[List[str]] = None
    max_retries: int = 2

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        last_message = messages[-1]
        print(type(last_message))
        tokens = last_message.content[: self.parrot_buffer_length]
        ct_input_tokens = sum(len(message.content) for message in messages)
        ct_output_tokens = len(tokens)
        message = AIMessage(
            content=tokens,
            additional_kwargs={},  # Used to add additional payload to the message
            response_metadata={  # Use for response metadata
                "time_in_seconds": 3,
                "model_name": self.model_name,
            },
            usage_metadata={
                "input_tokens": ct_input_tokens,
                "output_tokens": ct_output_tokens,
                "total_tokens": ct_input_tokens + ct_output_tokens,
            },
        )
        ##

        generation = ChatGeneration(message=message)
        return ChatResult(generations=[generation])

    def _stream(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> Iterator[ChatGenerationChunk]:
        last_message = messages[-1]
        tokens = str(last_message.content[: self.parrot_buffer_length])
        ct_input_tokens = sum(len(message.content) for message in messages)

        for token in tokens:
            usage_metadata = UsageMetadata(
                {
                    "input_tokens": ct_input_tokens,
                    "output_tokens": 1,
                    "total_tokens": ct_input_tokens + 1,
                }
            )
            ct_input_tokens = 0
            chunk = ChatGenerationChunk(
                message=AIMessageChunk(content=token, usage_metadata=usage_metadata)
            )

            if run_manager:
                # This is optional in newer versions of LangChain
                # The on_llm_new_token will be called automatically
                run_manager.on_llm_new_token(token, chunk=chunk)

            yield chunk

        # Let's add some other information (e.g., response metadata)
        chunk = ChatGenerationChunk(
            message=AIMessageChunk(
                content="",
                response_metadata={"time_in_sec": 3, "model_name": self.model_name},
            )
        )
        if run_manager:
            # This is optional in newer versions of LangChain
            # The on_llm_new_token will be called automatically
            run_manager.on_llm_new_token(token, chunk=chunk)
        yield chunk

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model."""
        return "echoing-chat-model-advanced"

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters.

        This information is used by the LangChain callback system, which
        is used for tracing purposes make it possible to monitor LLMs.
        """
        return {
            # The model name allows users to specify custom token counting
            # rules in LLM monitoring applications (e.g., in LangSmith users
            # can provide per token pricing for their model and monitor
            # costs for the given LLM.)
            "model_name": self.model_name,
        }
    
    def bind_tools(
            self, 
            tools: Sequence[Union[dict[str, Any], type, Callable, BaseTool]],
            *,
            tool_choice: Optional[
                Union[dict, str, Literal["auto", "none", "required", "any"], bool]
            ] = None,
            strict: Optional[bool] = None,
            parallel_tool_calls: Optional[bool] = None,
            **kwargs: Any,
        ) -> Runnable[LanguageModelInput, BaseMessage]:
        formatted_tools = [
            convert_to_openai_tool(tool, strict=strict) for tool in tools
        ]
        print(formatted_tools)

        return super().bind(tools=formatted_tools, **kwargs)

In [31]:
model = ChatParrotLink(parrot_buffer_length=3, model="my_custom_model")

model.invoke(
    [
        HumanMessage(content="hello!"),
        AIMessage(content="Hi there human!"),
        HumanMessage(content="Meow!"),
    ]
)

<class 'langchain_core.messages.human.HumanMessage'>


AIMessage(content='Meo', additional_kwargs={}, response_metadata={'time_in_seconds': 3, 'model_name': 'my_custom_model'}, id='run-c0928052-46af-4fa4-92f9-d02a9afcef78-0', usage_metadata={'input_tokens': 26, 'output_tokens': 3, 'total_tokens': 29})

In [32]:
model.invoke("hello")

<class 'langchain_core.messages.human.HumanMessage'>


AIMessage(content='hel', additional_kwargs={}, response_metadata={'time_in_seconds': 3, 'model_name': 'my_custom_model'}, id='run-86846f81-cfd3-4ac8-acdf-664bb6f6fe4f-0', usage_metadata={'input_tokens': 5, 'output_tokens': 3, 'total_tokens': 8})

In [33]:
model.batch(["hello", "goodbye"])

<class 'langchain_core.messages.human.HumanMessage'><class 'langchain_core.messages.human.HumanMessage'>



[AIMessage(content='hel', additional_kwargs={}, response_metadata={'time_in_seconds': 3, 'model_name': 'my_custom_model'}, id='run-eaa3f2cb-d1a1-4794-b9fd-723e70e32d19-0', usage_metadata={'input_tokens': 5, 'output_tokens': 3, 'total_tokens': 8}),
 AIMessage(content='goo', additional_kwargs={}, response_metadata={'time_in_seconds': 3, 'model_name': 'my_custom_model'}, id='run-c8bf4b9e-ac04-412d-bc6a-f500e48d44b5-0', usage_metadata={'input_tokens': 7, 'output_tokens': 3, 'total_tokens': 10})]

In [34]:
for chunk in model.stream("horse"):
    print(chunk.content, end="|")

h|o|r||

In [35]:
async for chunk in model.astream("cat"):
    print(chunk.content, end="|")

c|a|t||

In [36]:
async for event in model.astream_events("cat", version="v1"):
    print(event)

{'event': 'on_chat_model_start', 'run_id': '7df3045a-d611-41af-9f5e-56a8f2935535', 'name': 'ChatParrotLink', 'tags': [], 'metadata': {}, 'data': {'input': 'cat'}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': '7df3045a-d611-41af-9f5e-56a8f2935535', 'tags': [], 'metadata': {}, 'name': 'ChatParrotLink', 'data': {'chunk': AIMessageChunk(content='c', additional_kwargs={}, response_metadata={}, id='run-7df3045a-d611-41af-9f5e-56a8f2935535', usage_metadata={'input_tokens': 3, 'output_tokens': 1, 'total_tokens': 4})}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': '7df3045a-d611-41af-9f5e-56a8f2935535', 'tags': [], 'metadata': {}, 'name': 'ChatParrotLink', 'data': {'chunk': AIMessageChunk(content='a', additional_kwargs={}, response_metadata={}, id='run-7df3045a-d611-41af-9f5e-56a8f2935535', usage_metadata={'input_tokens': 0, 'output_tokens': 1, 'total_tokens': 1})}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': '7df3045a-d611-41af-9f5e-56a8f29355

In [37]:
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults

load_dotenv()
tool = TavilySearchResults(max_results = 3)
tools = [tool]

# llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = model.bind_tools(tools)

[{'type': 'function', 'function': {'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]


In [39]:
llm_with_tools.model_dump()

{'name': None,
 'bound': {'name': None,
  'disable_streaming': False,
  'model_name': 'my_custom_model',
  'parrot_buffer_length': 3,
  'temperature': None,
  'max_tokens': None,
  'timeout': None,
  'stop': None,
  'max_retries': 2},
 'kwargs': {'tools': [{'type': 'function',
    'function': {'name': 'tavily_search_results_json',
     'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.',
     'parameters': {'properties': {'query': {'description': 'search query to look up',
        'type': 'string'}},
      'required': ['query'],
      'type': 'object'}}}]},
 'config': {},
 'config_factories': [],
 'custom_input_type': None,
 'custom_output_type': None}

In [38]:
llm_with_tools.invoke("3+2=?")

<class 'langchain_core.messages.human.HumanMessage'>


AIMessage(content='3+2', additional_kwargs={}, response_metadata={'time_in_seconds': 3, 'model_name': 'my_custom_model'}, id='run-262b0da8-9766-424d-bd1d-aab3ecd6f587-0', usage_metadata={'input_tokens': 5, 'output_tokens': 3, 'total_tokens': 8})

In [20]:
from typing import Annotated
from langchain.agents import tool

@tool
def add_function(a: Annotated[float, "first number to add"], b: Annotated[float, "second number to add"]) -> float:
    """Adds two numbers together."""
    return a + b

In [24]:
hasattr(add_function, "model_json_schema")

True

In [ ]:
from langchain_core.utils.function_calling import (
    convert_to_openai_function,
    convert_to_openai_tool,
)

convert_to_openai_tool(add_function)

{'type': 'function',
 'function': {'name': 'add_function',
  'description': 'Adds two numbers together.',
  'parameters': {'properties': {'a': {'description': 'first number to add',
     'type': 'number'},
    'b': {'description': 'second number to add', 'type': 'number'}},
   'required': ['a', 'b'],
   'type': 'object'}}}

In [21]:
add_function.model_dump()

{'name': 'add_function',
 'description': 'Adds two numbers together.',
 'args_schema': langchain_core.utils.pydantic.add_function,
 'return_direct': False,
 'verbose': False,
 'tags': None,
 'metadata': None,
 'handle_tool_error': False,
 'handle_validation_error': False,
 'response_format': 'content',
 'func': <function __main__.add_function(a: typing.Annotated[float, 'first number to add'], b: typing.Annotated[float, 'second number to add']) -> float>,
 'coroutine': None}

In [ ]:
add_function.args

{'a': {'description': 'first number to add', 'title': 'A', 'type': 'number'},
 'b': {'description': 'second number to add', 'title': 'B', 'type': 'number'}}

In [13]:
!pip install -qU langchain-teddynote

  DEPRECATION: sgmllib3k is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  DEPRECATION: kiwipiepy_model is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain_teddynote.tools.tavily import TavilySearch
from langchain.agents import create_tool_calling_agent